In [175]:
# Follow instructions from https://github.com/AierLab/indl-dataset to generate data

In [176]:
# Import packages
import os
import pandas as pd
import matplotlib as plt
import timm
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import random_split, DataLoader
from torch.utils.data import Subset 
from PIL import Image
from torch.utils.data.dataloader import default_collate

#from scripts import Worker

In [177]:
# Set wd

path_to_set = "/Users/leery/Documents/EllaNB240/indl-dataset-main" # paste path here
os.chdir(path_to_set)
print("Current Working Directory:", os.getcwd())

Current Working Directory: /Users/leery/Documents/EllaNB240/indl-dataset-main


In [178]:
# # Generate data
# !python data/data_generate.py

In [179]:
# CONFIG #
# Adapted from Assignment 2 codebase

# TRAINING SETTINGS
NUM_EPOCHS = 5


# LEARNING RATE SETTINGS
BASE_LR = 0.001
DECAY_WEIGHT = 0.1 # factor by which the learning rate is reduced.
EPOCH_DECAY = 30 # number of epochs after which the learning rate is decayed exponentially by DECAY_WEIGHT.


# DATASET INFO
NUM_CLASSES = 2 # set the number of classes in your dataset
DATA_DIR = 'InDL' # to run with the sample dataset, just set to 'hymenoptera_data'


# DATALOADER PROPERTIES
BATCH_SIZE = 5 # originally 10


# GPU SETTINGS
#TORCH_DEVICE = 'mps:0' # Enter device ID of your gpu if you want to run on gpu. Otherwise neglect.
device = torch.device("mps")
GPU_MODE = 1 # set to 1 if want to run on gpu.


# SETTINGS FOR DISPLAYING ON TENSORBOARD
USE_TENSORBOARD = 0 #if you want to use tensorboard set this to 1.
TENSORBOARD_SERVER = "YOUR TENSORBOARD SERVER ADDRESS HERE" # If you set.
EXP_NAME = "fine_tuning_experiment" # if using tensorboard, enter name of experiment you want it to be displayed as.

In [219]:
# Define transformations###

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))], transforms.Resize((img_size, img_size))) 

# classes = (0, 1)
# # model_names = ['xception','densenet201','resnetv2_50', 'vgg16', 'darknet53', 'mobilenetv3_large_100', 'inception_resnet_v2', 'nasnetalarge', 'efficientnetv2_rw_m', 'convnext_xlarge_384_in22ft1k']
# model_names = []

NameError: name 'img_size' is not defined

In [181]:
# Define dataset class
class MyDataset(Dataset):
    def __init__(self, data_dir, transforms=None):
        self.data_dir = data_dir
        self.data_info = self.get_img_info(data_dir)
        self.transforms = transforms

    def __getitem__(self, item):
        path_img, label = self.data_info.iloc[item][1:3]
        label = int(label)
        path_img = os.path.join(self.data_dir, path_img)
        image = Image.open(path_img).convert('RGB') # Gray scale is enough for logic interpretation
        # 使用定义好的transforms，对数据进行处理
        if self.transforms is not None:
            image = self.transforms(image)

        return image, label

    def __len__(self):
        return len(self.data_info)
    
    def get_img_info(self, data_dir):
        path_dir = os.path.join(data_dir, 'label.csv')
        return pd.read_csv(path_dir)

In [182]:
# Create a function to split training and val data in each folder

def train_val_split(folder):
    full_train = MyDataset(f"/Users/leery/Documents/EllaNB240/indl-dataset-main/train/{dataset_folder}", transform)
    train_indices, val_indices = random_split(range(len(full_train)), [1200, 400]) 
    train_set = Subset(full_train, train_indices) 
    val_set = Subset(full_train, val_indices) # Store in dictionary 
    dsets = {'train': train_set, 'val': val_set}

    return dsets

In [183]:
# Define train and val datasets
separate_data = [] # Define empty list to store dicts for each folder

for dataset_folder in ["dataset01", "dataset02", "dataset03", "dataset04", "dataset05"]:
    separate_data.append(train_val_split(dataset_folder))

In [214]:
# # Combine data into train and val datasets

dsets = {}

for i in range(len(separate_data)):
#     dsets = {'train': separate_data[i-1]['train'], 'val': separate_data[i-1]['val']}
#     new_items = {'train': separate_data[i]['train'], 'val': separate_data[i]['val']}
#     print(new_items)
#     dsets.update(new_items)

    #dsets.setdefault('train', []).append(separate_data[i-1]['train'])
    dsets.setdefault('train', []).extend(separate_data[i]['train'])

    #dsets.setdefault('val', []).append(separate_data[i-1]['val'])
    dsets.setdefault('val', []).extend(separate_data[i]['val'])

In [215]:
# Get dataset sizes
dset_sizes = {'train': len(dsets['train']), 'val': len(dsets['val'])}

# Define DataLoaders
dset_loaders = {
    'train': DataLoader(dsets['train'], batch_size=BATCH_SIZE, shuffle=True, num_workers=0, collate_fn=default_collate),
    'val': DataLoader(dsets['val'], batch_size=BATCH_SIZE, shuffle=False, num_workers=0, collate_fn=default_collate)
}

In [186]:
# Import pertinent packages
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import time
import copy
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [187]:
# Define GPU use
use_gpu = GPU_MODE

In [188]:
# Define model (TRYING SAME AS CIFAR-10 FOR NOW)
# Source: https://www.kaggle.com/code/datascienceimcomming/cifar10-with-cnn-90-accuracy

class SimpleResidualBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3, 
            out_channels=3, 
            kernel_size=3, 
            stride=1, 
            padding=1
        )
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(
            in_channels=3, 
            out_channels=3, 
            kernel_size=3, 
            stride=1, 
            padding=1
        )
        self.relu2 = nn.ReLU()
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        return self.relu2(out).to(device) + x

def conv_block(in_channels, out_channels, pool=False):
    layers = [
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), 
        nn.BatchNorm2d(out_channels), 
        nn.ReLU(inplace=True)
    ]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class Model_CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(
            conv_block(128, 128), 
            conv_block(128, 128)
        )
        
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(
            conv_block(512, 512), 
            conv_block(512, 512)
        )
        
        self.classifier = nn.Sequential(
            nn.MaxPool2d(4), 
            nn.Flatten(), 
            nn.Linear(512, num_classes)
        )
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out.to(device)

In [189]:
# Define function to train the model

def train_model(model, criterion, optimizer, lr_scheduler, num_epochs=100):
    since = time.time()

    best_model = model.to(device)
    best_acc = 0.0

    accuracies = {'train': [], 'val': []}
    losses = {'train': [], 'val': []}
    for epoch in range(num_epochs):
        print('-' * 10)
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                optimizer = lr_scheduler(optimizer, epoch)
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to eval mode for validation (no need to track gradients)

            running_loss = 0.0
            running_corrects = 0

            counter=0
            # Iterate over data, getting one batch of inputs (images) and labels each time.
            for inputs, labels in dset_loaders[phase]:
                #inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                #print(f"Labels shape: {labels.shape}, Labels: {labels}, Inputs: {inputs}")

                if use_gpu:

                    try:
                        inputs, labels = inputs.to(device), labels.to(device)
                    except Exception as e:
                        print("ERROR! here are the inputs and labels before we print the full stack trace:")
                        print(inputs, labels)
                        raise e
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # Set gradient to zero to delete history of computations in previous epoch. Track operations so that differentiation can be done automatically.
                optimizer.zero_grad()
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)

                loss = criterion(outputs, labels)

                # Print a line every 10 batches so you have something to watch and don't feel like the program isn't running.
                if counter%10==0:
                    print("Reached batch iteration", counter)

                counter+=1

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                try:
                    running_loss += loss.item()
                    running_corrects += torch.sum(preds == labels.data)
                except:
                    print('unexpected error, could not calculate loss or do a sum.')

            epoch_loss = running_loss / dset_sizes[phase]
            epoch_acc = running_corrects.item() / float(dset_sizes[phase])
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            accuracies[phase].append(epoch_acc)
            losses[phase].append(epoch_loss)

            # deep copy the model
            if phase == 'val':
                if USE_TENSORBOARD:
                    foo.add_scalar_value('epoch_loss', epoch_loss,step=epoch)
                    foo.add_scalar_value('epoch_acc', epoch_acc,step=epoch)
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model = copy.deepcopy(model)
                    print('new best accuracy =', best_acc)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    print('returning and looping back')

    return best_model, accuracies, losses

In [190]:
# # Define function for modifying learning rate

def exp_lr_scheduler(optimizer, epoch, init_lr=BASE_LR, lr_decay_epoch=EPOCH_DECAY):
    """Decay learning rate by a factor of DECAY_WEIGHT every lr_decay_epoch epochs."""
    lr = init_lr * (DECAY_WEIGHT**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

In [191]:
# Define model architecture
model_ft = models.resnet50(pretrained=True)

criterion = nn.CrossEntropyLoss()

if use_gpu:
    criterion.to(device)
    model_ft.to(device)

optimizer_ft = optim.RMSprop(model_ft.parameters(), lr=0.0001)

/Users/leery/miniconda3/envs/dalgi/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/leery/miniconda3/envs/dalgi/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [209]:
# Run the functions and save the best model in the function model_ft.
model_ft, accuracies, losses = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=NUM_EPOCHS)

----------
Epoch 0/4
----------
LR is set to 0.001


RuntimeError: stack expects each tensor to be equal size, but got [3, 256, 256] at entry 0 and [3, 128, 128] at entry 1

In [ ]:
# Save model
# batch 5 epoch 5
# torch.save(model_ft.state_dict(), 'InDL_train_val.pt')

# batch 15 epoch 10
#torch.save(model_ft_2.state_dict(), 'InDL_train_val_2.pt')

# batch 30 epoch 10
torch.save(model_ft_4.state_dict(), 'InDL_train_val_4.pt')